In [1]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, AffectedPayload
from models.Source import DataSource as SecApi
from utils.transforms import deltatime_fmt, str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"{"usuarios regionales":<22}",':',f"{clients:,.0f} clientes")

#data
curr = SecApi()
curr.request(source='affected_detail',payload=AffectedPayload()) 
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
print(curr_df.info())
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print(f'{'usuarios sin servicio':<22}',':',f'{affected:,.0f}',' clientes',f'({affected/clients:.2%})')
print(f'{'fecha reporte':<22}',':',curr.report_date.strftime("%d/%m/%Y %H:%M"))

res:DataFrame = curr_df\
    .groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
    
res['TIEMPO_SIN_SERVICIO'] = res['FECHA_INT_STR'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=0))

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(deltatime_fmt)
res

usuarios regionales    : 780,563 clientes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DIA_INT             569 non-null    int64         
 1   MES_INT             569 non-null    int64         
 2   ANHO_INT            569 non-null    int64         
 3   HORA                569 non-null    int64         
 4   DIA                 569 non-null    int64         
 5   MES                 569 non-null    int64         
 6   ANHO                569 non-null    int64         
 7   NOMBRE_REGION       569 non-null    object        
 8   NOMBRE_COMUNA       569 non-null    object        
 9   NOMBRE_EMPRESA      569 non-null    object        
 10  CLIENTES_AFECTADOS  569 non-null    int64         
 11  ACTUALIZADO_HACE    569 non-null    object        
 12  FECHA_INT_STR       569 non-null    datetime64[ns]
dtypes: datet

,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
6,Viña del Mar,CHILQUINTA,2024-02-19,155,190 dias 10 horas
3,Quilpue,CHILQUINTA,2024-02-19,37,190 dias 10 horas
4,Valparaiso,CGE,2024-08-27,2,0 dias 10 horas
5,Viña del Mar,CGE,2024-08-27,2,0 dias 10 horas
0,Cabildo,CGE,2024-08-27,1,0 dias 10 horas
1,Papudo,CGE,2024-08-27,1,0 dias 10 horas
2,Puchuncavi,CGE,2024-08-27,1,0 dias 10 horas


In [2]:
res_by_city:DataFrame = curr_df.groupby(['NOMBRE_COMUNA'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res_by_city

,NOMBRE_COMUNA,CLIENTES_AFECTADOS
5,Viña del Mar,157
3,Quilpue,37
4,Valparaiso,2
0,Cabildo,1
1,Papudo,1
2,Puchuncavi,1


In [19]:
from models.Payload import Payload,AffectedPayload
from models.Source import DataSource as Api
from utils.transforms import str2date,deltatime_fmt
from pandas import DataFrame
#cspell:disable
api = Api()
delay_folder = [{'delay_hours':0},{'delay_hours':1},{'delay_hours':6},{'delay_hours':12},{'delay_hours':24}]
data_folder = []
for delay in delay_folder:
    api.request(source='affected_detail',payload=AffectedPayload(**delay))
    result:DataFrame = api.dataframe(FECHA_INT_STR=str2date).copy()
    ## transform
    ### rename
    result = result.rename(columns={
        "DIA_INT":"día",
        "MES_INT":"mes",
        "ANHO_INT":"año",
        "NOMBRE_REGION":"región",
        "NOMBRE_COMUNA":"comuna",
        "NOMBRE_EMPRESA":"distribuidora",
        "ACTUALIZADO_HACE":"actualización",
        "FECHA_INT_STR":"fecha incidente",
        "CLIENTES_AFECTADOS":"afectados",
        })

    ###calculate columns
    result['periodo sin servicio']=result['fecha incidente'].apply(deltatime_fmt)

    ###remove
    result = result.drop(columns=['HORA','DIA','MES','ANHO'])
    
    ###append
    data_folder.append(result)

print(data_folder)

[     día  mes  ...  fecha incidente periodo sin servicio
0     27    8  ...       2024-08-27      0 dias 15 horas
1     27    8  ...       2024-08-27      0 dias 15 horas
2     27    8  ...       2024-08-27      0 dias 15 horas
3     27    8  ...       2024-08-27      0 dias 15 horas
4     26    8  ...       2024-08-26      1 dias 15 horas
..   ...  ...  ...              ...                  ...
693   26    8  ...       2024-08-26      1 dias 15 horas
694   27    8  ...       2024-08-27      0 dias 15 horas
695   27    8  ...       2024-08-27      0 dias 15 horas
696   27    8  ...       2024-08-27      0 dias 15 horas
697   27    8  ...       2024-08-27      0 dias 15 horas

[698 rows x 10 columns],      día  mes  ...  fecha incidente periodo sin servicio
0     27    8  ...       2024-08-27      0 dias 15 horas
1     25    8  ...       2024-08-25      2 dias 15 horas
2     27    8  ...       2024-08-27      0 dias 15 horas
3     27    8  ...       2024-08-27      0 dias 15 horas
4   